In [26]:
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, log_loss

In [12]:
data = pd.read_csv('/home/maksonvinicio/Documents/GitHub/Data-Science-Projects/Stroke Prediction/data/healthcare-dataset-stroke-data.csv')

data = data.drop(['id'], axis=1)

In [13]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [190]:
X = data.drop(['stroke'], axis=1)
y = data['stroke']


ct = ColumnTransformer(transformers=[('ohe', OneHotEncoder(), ['gender', 'work_type', 'smoking_status'])],
                       remainder='passthrough')
X = ct.fit_transform(X)


colunas = ['ohe1', 'ohe2', 'ohe3', 'ohe4', 'ohe5', 'ohe6', 'ohe7', 'ohe8', 'ohe9', 'ohe10', 'ohe11', 'ohe12',
           'age', 'hypertension', 'heart_disease', 'ever_married', 'Residence_type',
           'avg_glucose_level', 'bmi']

X = pd.DataFrame(X, columns=colunas)


X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.4, random_state=42)


In [191]:
cols = ['ever_married', 'Residence_type'] #unique pra descobrir quantos rótulos únicos
le = LabelEncoder()
X_train[cols] = X_train[cols].apply(le.fit_transform)
X_val[cols] = X_val[cols].apply(le.fit_transform)

X_train.head(2)

/home/maksonvinicio/.local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,ohe1,ohe2,ohe3,ohe4,ohe5,ohe6,ohe7,ohe8,ohe9,ohe10,ohe11,ohe12,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi
3563,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,64.0,0,1,1,0,109.88,33.9
3245,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,79.0,1,0,1,0,64.44,26.9


In [192]:
from sklearn.preprocessing import StandardScaler
cols_scaler = ['age', 'avg_glucose_level', 'bmi']
sc = StandardScaler()
X_train[cols_scaler] = sc.fit_transform(X_train[cols_scaler])
X_val[cols_scaler] = sc.transform(X_val[cols_scaler])

/home/maksonvinicio/.local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/maksonvinicio/.local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [198]:
X_train.isna().sum()

ohe1                   0
ohe2                   0
ohe3                   0
ohe4                   0
ohe5                   0
ohe6                   0
ohe7                   0
ohe8                   0
ohe9                   0
ohe10                  0
ohe11                  0
ohe12                  0
age                    0
hypertension           0
heart_disease          0
ever_married           0
Residence_type         0
avg_glucose_level      0
bmi                  117
dtype: int64

In [202]:
from sklearn.ensemble import ExtraTreesRegressor

estimator = ExtraTreesRegressor(n_estimators=10, random_state=42)
it_imp = IterativeImputer(estimator=estimator, missing_values=np.nan, max_iter=10)

X_train['bmi'] = it_imp.fit_transform(X_train['bmi'].values.reshape(-1, 1))
X_val['bmi'] = it_imp.fit_transform(X_val['bmi'].values.reshape(-1, 1))

/home/maksonvinicio/.local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/maksonvinicio/.local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
